# 🚀 Model Training

This notebook guides you through the process of training a policy on your converted dataset. 

The process is broken down into a few simple steps:
1.  **Setup**: Apply necessary patches to the `lerobot` library.
2.  **Dataset**: Specify the path to your training data.
3.  **Configuration**: Select a model architecture and its hyperparameters.
4.  **Training**: Launch the training process.

--- 
### 1. Setup

First, apply our custom patches to the `lerobot` library. This only needs to be done once per session.

In [1]:
# Set environment variables BEFORE any imports
import os
import warnings

os.environ["LEROBOT_VIDEO_BACKEND"] = "pyav"

# Suppress torchvision video deprecation warning
warnings.filterwarnings("ignore", message=".*video decoding and encoding capabilities.*")

from example_policies import lerobot_patches

lerobot_patches.apply_patches()

/home/yizhang/miniconda3/envs/hackathon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- 
### 2. Select Dataset

> **Action Required:** Update `DATA_DIR` to point to the dataset you created in the previous notebook.

In [2]:
import pathlib

# TODO: Set the path to your converted dataset directory.
DATA_DIR = pathlib.Path("/home/yizhang/Projects/hackathon-ki-fabrik/data/sort_red_blocks_80_old")

--- 
### 3. Select Model Configuration

We provide several pre-made configurations as a starting point, but recommend using dit_flow_config. You can also adjust parameters like `batch_size` as needed.

In [3]:
# Select one of the following configurations
from example_policies.config_factory import diffusion_config, dit_flow_config

cfg = dit_flow_config(DATA_DIR, enable_wandb=False, policy_kwargs={"image_only": True})

# Disable multiprocessing if there are issues with dataloader workers
cfg.num_workers = 64


📊 Training by epochs:
   - Dataset size: 26800 frames
   - Batch size: 64
   - Epochs: 200
   - Calculated steps: 83600
   - Save every: 100 epochs (41800 steps)

Final Training Configuration (full details):
TrainPipelineConfig(dataset=DatasetConfig(repo_id='sort_red_blocks_80_old',
                                          root=PosixPath('/home/yizhang/Projects/hackathon-ki-fabrik/data/sort_red_blocks_80_old'),
                                          episodes=[0,
                                                    1,
                                                    2,
                                                    3,
                                                    4,
                                                    5,
                                                    6,
                                                    7,
                                                    8,
                                                    9,
                                 

In [4]:
# cfg.log_freq = 1
cfg.save_freq = 10000
cfg.steps = 80000


cfg.policy.optimizer_lr = 2e-4

cfg.job_name = "ditflow_sort_red_blocks_80"
cfg.output_dir = pathlib.Path("/home/yizhang/Projects/hackathon-ki-fabrik/outputs/ditflow_sort_red_blocks_80")
cfg.wandb.enable = True
cfg.wandb.project = "paper"
cfg.wandb.entity = "470620104-technical-university-of-munich"

import json

print(json.dumps(cfg.to_dict(), indent=4))

{
    "dataset": {
        "repo_id": "sort_red_blocks_80_old",
        "root": "/home/yizhang/Projects/hackathon-ki-fabrik/data/sort_red_blocks_80_old",
        "episodes": [
            0,
            1,
            2,
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18,
            19,
            20,
            21,
            22,
            23,
            24,
            25,
            26,
            27,
            28,
            29,
            30,
            31,
            32,
            33,
            34,
            35,
            36,
            37,
            38,
            39,
            40,
            41,
            42,
            43,
            44,
            45,
            46,
            47,
            48,
            49,
            50,
            51,
  

You can specify additional keywords by looking at the lerobot configuration code, e.g. `lerobot.policies.act.configuration_act`
and then adapt the code cell accordingly:
```python
cfg = act_config(DATA_DIR, policy_kwargs={
    optimizer_lr=1e-5
})
```

--- 
### 4. Start Training

This cell will start the training process. Metrics and logs will be streamed to the console, and if you have configured it, to Weights & Biases.

In [5]:
from example_policies.train import train

# Set video backend in config
cfg.dataset.video_backend = "pyav"

train(cfg)

INFO 2025-12-03 14:37:42 ts/train.py:111 {'batch_size': 64,
 'dataset': {'episodes': [0,
                          1,
                          2,
                          3,
                          4,
                          5,
                          6,
                          7,
                          8,
                          9,
                          10,
                          11,
                          12,
                          13,
                          14,
                          15,
                          16,
                          17,
                          18,
                          19,
                          20,
                          21,
                          22,
                          23,
                          24,
                          25,
                          26,
                          27,
                          28,
                          29,
                          30,
                    


Starting training...


INFO 2025-12-03 14:37:43 _patches.py:128 Track this run --> https://wandb.ai/470620104-technical-university-of-munich/paper/runs/73lsqwnu
INFO 2025-12-03 14:37:43 ts/train.py:127 Creating dataset


Logs will be synced with wandb.


INFO 2025-12-03 14:37:44 ts/train.py:138 Creating policy
INFO 2025-12-03 14:37:44 ts/train.py:144 Creating optimizer and scheduler
INFO 2025-12-03 14:37:44 ts/train.py:156 Output dir: /home/yizhang/Projects/hackathon-ki-fabrik/outputs/ditflow_sort_red_blocks_80
INFO 2025-12-03 14:37:44 ts/train.py:159 cfg.steps=80000 (80K)
INFO 2025-12-03 14:37:44 ts/train.py:160 dataset.num_frames=21308 (21K)
INFO 2025-12-03 14:37:44 ts/train.py:161 dataset.num_episodes=82
INFO 2025-12-03 14:37:44 ts/train.py:162 num_learnable_params=75717923 (76M)
INFO 2025-12-03 14:37:44 ts/train.py:163 num_total_params=75718173 (76M)
INFO 2025-12-03 14:37:44 ts/train.py:202 Start offline training on a fixed dataset


Number of flow params: 42.13M


INFO 2025-12-03 14:42:39 ts/train.py:232 step:200 smpl:13K ep:49 epch:0.60 loss:0.892 grdn:0.885 lr:4.0e-05 updt_s:0.237 data_s:1.236
INFO 2025-12-03 14:47:05 ts/train.py:232 step:400 smpl:26K ep:99 epch:1.20 loss:0.362 grdn:0.826 lr:1.2e-04 updt_s:0.231 data_s:1.100
INFO 2025-12-03 14:51:20 ts/train.py:232 step:600 smpl:38K ep:148 epch:1.80 loss:0.232 grdn:0.678 lr:1.9e-04 updt_s:0.227 data_s:1.048
INFO 2025-12-03 14:55:36 ts/train.py:232 step:800 smpl:51K ep:197 epch:2.40 loss:0.173 grdn:0.615 lr:2.0e-04 updt_s:0.227 data_s:1.048
INFO 2025-12-03 14:59:51 ts/train.py:232 step:1K smpl:64K ep:246 epch:3.00 loss:0.145 grdn:0.654 lr:2.0e-04 updt_s:0.227 data_s:1.049
INFO 2025-12-03 15:04:05 ts/train.py:232 step:1K smpl:77K ep:296 epch:3.60 loss:0.128 grdn:0.603 lr:2.0e-04 updt_s:0.227 data_s:1.046
INFO 2025-12-03 15:08:22 ts/train.py:232 step:1K smpl:90K ep:345 epch:4.20 loss:0.117 grdn:0.610 lr:2.0e-04 updt_s:0.227 data_s:1.055
INFO 2025-12-03 15:12:38 ts/train.py:232 step:2K smpl:102K e

In [ ]:
# upload policy to hub

!hf upload --repo-type model continuallearning/ditflow_sort_red_blocks_80 /home/yizhang/Projects/hackathon-ki-fabrik/outputs/ditflow_sort_red_blocks_80/checkpoints/last/pretrained_model

Start hashing 4 files.
Finished hashing 4 files.
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

  ...d_model/model.safetensors:   1%|              | 2.52MB /  303MB            

Processing Files (0 / 1)      :   1%|              | 2.52MB /  303MB, 6.31MB/s  

Processing Files (0 / 1)      :   3%|▍             | 8.38MB /  303MB, 14.0MB/s  

Processing Files (0 / 1)      :   3%|▍             | 9.74MB /  303MB, 12.2MB/s  
New Data Upload               :   1%|              |  686kB /  134MB,  857kB/s  

Processing Files (0 / 1)      :   3%|▍             | 10.4MB /  303MB, 10.4MB/s  
New Data Upload               :   1%|▏             | 1.37MB /  134MB, 1.37MB/s  

Processing Files (0 / 1)      :   4%|▌             | 13.4MB /  303MB, 11.2MB/s  
New Data Upload               :   2%|▏             | 3.34MB /  201MB, 2.79MB/s  

Processing Files (0 / 1)      :   6%|▊             | 

: 